# Nematic Order Statistics

## Run many simulations in parallel

In [8]:
import lammps2d as lmp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anm
import matplotlib.collections as clt
import pandas as pd
import string as st
import scipy.spatial as spp
import scipy.optimize as spo
import sympy as smp
import copy as cp
import pickle
import os
import sys
import datetime
import getpass
import pysftp
from IPython.display import HTML

smp.init_printing()
%matplotlib inline
%reload_ext autoreload
%autoreload 2

The previous notebook results in a very noisy order parameter, where no clear trend of $S$ vs $B$ can be extracted. It's true that the final configuration depends on the intitial conditions which are all the same for our system, but there is also a stochastic contribution. This means that several realizations of the same test will result in varying order paramters, and we then need some statistics. 

In this notebook I make several experiments of the same parameters as before, which are the parameters that Dobnikar predicts as dimer forming. Since the experimetns take a while, I run them in parallel in six of the eight threads of a locally configured cluster. 

Configuring a local cluster is very easy with ipycluster. ipycluster can be installed using conda, and then we can run in DOS:

    !ipcluster start -n 6

which starts six parallel processes. 

The previous run of this notebook took conspicuosly little time. I have now realized that the reason this happened is that the packing was too low. Low packing simulations are faster because there are less particles within the cutoff which makes for less calculations. I must run this again. 

If the simulations take about 25 minutes, and I run 10 experiments with 20 fields, in six parallel threads, the total time will be around 13 hours.

In [9]:
25*10*20/6/60

This notebook was run in the lab computer. From the report of the finished simulations, it appears as if the whole process took about three hours. This is strange because it means that each simulation takes about 5 minute. In fact, these simulations take about 5 minutes, which is strange beacuse they appear to have the same conditions as the simulations of nb 19-StaticFieldNematicPhase, and those take about 25 minutes.

# Setup Cluster

In [10]:
from IPython.display import clear_output
import ipyparallel as ipp
c = ipp.Client()
dview = c[:]

In [11]:
c[:].apply_sync(lambda : "Hello, World")

['Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World']

# Setup environment of cluster workers

In [13]:
height = [4.046]
packing = [0.437]
field = np.linspace(1,20,20)
experiments = np.arange(0,10)
random = np.arange(0,2)

H,P,F,E,R = np.meshgrid(height,packing,field,experiments,random)

height = H.flatten()
packing = P.flatten()
field = F.flatten()
experiment = E.flatten()
random = R.flatten()

i = range(np.size(experiment))

dview.push(dict(height=height,packing=packing,field=field,experiment=experiment,random=random))
dview.push(dict(pswd=getpass.getpass()))

········


<AsyncResult: _push>

In [14]:
dview.push(dict(run_sim=run_sim))

<AsyncResult: _push>

In [15]:
%%px
import lammps2d as lmp
import numpy as np
import pickle
import datetime
import time
import pysftp
import os

In [16]:
amr = dview.map_async(lambda x: run_sim(x), i)
while not amr.wait(1):
    # clear output because we are printing the whole thing each time
    # rather than keeping track of what we've consumed
    clear_output(wait=True)
    for stdout in amr.stdout:
        sys.stdout.write(stdout)

In [41]:
amr.error

[None, None, None, None]

## Define a function for running

In [6]:
def run_sim(i):
    angle = 0
    radius = 1.4
    region, initial_positions = lmp.initial_setup(n_of_particles = 150, packing=packing[i], height = height[i])
    
    if random[i]:
        
        sim = lmp.sim(initial_positions,
                      magnitude = 0,
                      file_name = "field_%u"%field[i],
                      dir_name = "C:/Users/aortiza/Desktop/NematicPhase_0/exp_%u/"%experiment[i],
                      radius = radius,
                      diffusion = 0.073,
                      frequency=0,
                      susceptibility = 0.4,
                      angle=angle,
                      framerate=1e0,
                      timestep=1e-4,
                      total_time = 30,
                      temperature = 300,
                      stamp_time = False,
                      space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})

        sim.generate_scripts()
        sim.run()

        trj = lmp.trj_lazyread(sim.base_name+".lammpstrj")[-1]
        initial_positions = trj.filter(('x','y','z')).values

    sim = lmp.sim(initial_positions,
              magnitude = field[i],
              file_name = "field_%u"%field[i],
              dir_name = "C:/Users/aortiza/Desktop/NematicPhase_0/exp_%u/"%experiment[i],
              radius = radius,
              diffusion = 0.073,
              frequency=0,
              susceptibility = 0.4,
              angle=angle,
              framerate=1e2,
              timestep=0.5e-4,
              total_time = 120,
              temperature = 300,
              stamp_time = False,
              space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})
    
    sim.generate_scripts()
    sim.run()
    
    trj = lmp.trj_lazyread(sim.base_name+".lammpstrj")[::100]
    lmp.export_animation(sim,trj,verb=False,start=0,end=sim.run_parameters.total_time,step=1,speedup=100)
    
    directory = 'Antonio/SimulationsOfDimerSystem/NematicPhase/DobnikarConditions_T300K/random_%u/exp_%u'%(random[i],experiment[i])

    cnopts = pysftp.CnOpts()
    cnopts.hostkeys = None
    with pysftp.Connection(host="161.116.81.190", username="aortiza", password=pswd, cnopts=cnopts) as magnetic:
        
        if not magnetic.isdir(directory):
            magnetic.makedirs(directory)

        with magnetic.cd(directory):
            magnetic.put(sim.base_name+'.lammpstrj')
            magnetic.put(sim.base_name+'.p')
            magnetic.put(sim.base_name+'.lmpin')
            magnetic.put(sim.base_name+'.gif')
    
    os.remove(sim.base_name+'.lammpstrj')
    os.remove(sim.base_name+'.p')
    os.remove(sim.base_name+'.lmpin')
    os.remove(sim.base_name+'.gif')